## Exploratory Analysis



In [1]:
%load_ext autoreload
%autoreload 2
import os

from glob import glob
import pandas as pd

In [2]:
import csv
import pandas as pd

train_df = pd.read_table("../data/tass2020/task2/train.tsv", index_col=0, names=["id", "text", "label"], quoting=csv.QUOTE_NONE)

dev_df = pd.read_table(
    "../data/tass2020/task2/dev.tsv", index_col=0,names=["id", "text", "label"],
    quoting=csv.QUOTE_NONE)

test_df = pd.read_table("../data/tass2020/task2/test_task2.tsv", index_col=0,names=["id", "text", "label"], quoting=csv.QUOTE_NONE)
 

train_df.sample(10)

,text,label
id,,
4974,Y a la vez para que haya científicas a las que...,others
852,"""Espero que los tiempos difíciles se han termi...",sadness
256,GRACIAS USER por unir a mi país contra el fasc...,joy
253,ARYA ERES LA PUTA AMA HASHTAG,joy
742,Que no termine el HASHTAG sin que nos permitái...,others
2386,¿De qué sirve que yo conciencie a mi alumnado ...,others
3943,"""El okupa ya puede ocupar tu casa El etarra pu...",others
2694,"Entiendo que existan gustos para todo, pero de...",joy
2907,"""HASHTAG """"Los propios aliados de USER están d...",fear


One label per tweet

In [3]:
full_df = pd.read_table(
    "../data/tass2020/task2/dataset_emotions_SP.csv",
    names=["id", "text", "label", "offensive"], 
    index_col=0,
    skiprows=1,
)

print(full_df.shape)

full_df["slice"] = "train"

full_df[:3]

(8409, 3)


,text,label,offensive,slice
id,,,,
1,Acabo de ver la gran pérdida que estamos tenie...,sadness,0,train
2,USER ¿A que vamos a reconstruir Notre Dame ant...,sadness,0,train
3,Desde ayer andan sufriendo por el incendio de ...,anger,0,train


Bueno, hacemos un train test nuestro. Es un quilombo partirlo de otra manera

In [24]:
not_found = []

text2id = {}
for idx, row in full_df.iterrows():
    text2id[row.text] = idx


In [25]:
import re 
from tqdm.auto import tqdm
from fuzzywuzzy import fuzz

url_regex = re.compile(
    r"((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|co|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"
)


hashtag_regex = re.compile(r'\B#(\w*[a-zA-Z]+\w*)')

for idx, row in tqdm(test_df.iterrows()):
    text = url_regex.sub("URL", row.text)
    text = hashtag_regex.sub("HASHTAG", row.text)
    found = []
    for other, idx in text2id.items():
        if (ratio := fuzz.ratio(text, other)) >= 92:
            found.append((other, ratio))
            full_df.loc[idx, "slice"] = "test"
            
    if len(found) != 1:
        print("="*80)
        print(text)
        print(found)

tweet
[]
AHORA | Comunicado Grupo de Lima sobre la situación en HASHTAG. "Conminan a Maduro a cesar la usurpación", se declaran en sesión permanente, y deciden reunirse de manera presencial el próximo viernes 3 de mayo, en la ciudad de Lima, Perú.  URL
[('AHORA | Comunicado Grupo de Lima sobre la situación en HASHTAG. "Conminan a Maduro a cesar la usurpación", se declaran en sesión permanente, y deciden reunirse de manera presencial el próximo viernes 3 de mayo, en la ciudad de Lima, Perú.  URL', 100), ('AHORA | Aquí el Comunicado del Grupo de Lima sobre la situación en HASHTAG. "Conminan a Maduro a cesar la usurpación", se declaran en sesión permanente, y deciden reunirse de manera presencial el próximo viernes 3 de mayo, en la ciudad de Lima, Perú.  URL', 98)]
ATENCIÓN CONTENIDO VIOLENTO-Un vehículo de la Guardia Nacional arrolla a manifestantes que le tiraban piedras en HASHTAG, HASHTAG. USER USER  URL
[('ATENCIÓN CONTENIDO VIOLENTO-Un vehículo de la Guardia Nacional arrolla a manif

In [26]:
full_df["slice"].value_counts()

train    6732
test     1677
Name: slice, dtype: int64

In [28]:
test_df

,text,label
id,,
id,tweet,NaN
1,El 26 de octubre de 2018 grabé este vídeo dura...,NaN
2,Se incendió HASHTAG en USER pese a todas las m...,NaN
3,Me da tristeza saber que fue incendiado y toda...,NaN
4,Hoy es un día triste en HASHTAG HASHTAG catedr...,NaN
...,...,...
1661,Se han dado cuenta que todos esos mentirosos c...,NaN
1662,USER USER Pocas cosas son tan imbéciles como l...,NaN
1663,Es imposible describir lo que se vive en HASHT...,NaN


Vamos con esto...

In [29]:
new_train_df = full_df[full_df["slice"] == "train"].copy() 
new_test_df = full_df[full_df["slice"] == "test"].copy()

del new_train_df["slice"]
del new_test_df["slice"]

new_test_df["label"].value_counts(normalize=True), train_df["label"].value_counts(normalize=True)


(others      0.493143
 joy         0.215862
 sadness     0.118664
 anger       0.101371
 surprise    0.040549
 disgust     0.019678
 fear        0.010733
 Name: label, dtype: float64,
 others       0.490742
 joy          0.215730
 sadness      0.119925
 anger        0.101919
 surprise     0.040938
 disgust      0.019195
 fear         0.011381
 label        0.000170
 Name: label, dtype: float64)

In [30]:
new_train_df.to_csv("../data/emoevent/train_es.csv")
new_test_df.to_csv("../data/emoevent/test_es.csv")

## Statistics


In [4]:
import pandas as pd

train_df = pd.read_csv("../data/emoevent/train_es.csv")
test_df = pd.read_csv("../data/emoevent/test_es.csv")



In [6]:
train_df["label"].value_counts()

others      3300
joy         1453
sadness      810
anger        687
surprise     276
disgust      128
fear          78
Name: label, dtype: int64

In [9]:
test_df["label"].value_counts()

others      827
joy         362
sadness     199
anger       170
surprise     68
disgust      33
fear         18
Name: label, dtype: int64

In [17]:
train_df["label"]

0       sadness
1       sadness
2         anger
3       sadness
4       disgust
         ...   
6727      anger
6728      anger
6729      anger
6730      anger
6731       fear
Name: label, Length: 6732, dtype: object

In [20]:
list(label2id), label2id

(['others', 'joy', 'sadness', 'anger', 'surprise', 'disgust', 'fear'],
 {'others': 0,
  'joy': 1,
  'sadness': 2,
  'anger': 3,
  'surprise': 4,
  'disgust': 5,
  'fear': 6})

In [18]:
from pysentimiento.emotion.datasets import id2label, label2id
from sklearn.utils.class_weight import compute_class_weight

compute_class_weight('balanced', list(label2id), y=train_df["label"])

/home/jmperez/.local/share/virtualenvs/pysent-oyXQVI9B/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=['others', 'joy', 'sadness', 'anger', 'surprise', 'disgust', 'fear'] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


array([ 0.29142857,  0.66188182,  1.18730159,  1.39987523,  3.48447205,
        7.51339286, 12.32967033])